*Este notebook contem material derivado do livro-texto e do repositório do autor: https://github.com/ageron/handson-ml*

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

# TensorFlow

TensorFlow é uma biblioteca para computação distribuída, incluindo GPUs. A biblioteca TensorFlow permite o processamento de **tensores** (matrizes n-dimensionais: um valor solitário é um tensor de grau zero, uma lista de valores é um tensor de grau 1, uma matriz é um tensor de grau 2, etc) através de um grafo de computação sobre o qual os dados **fluem** (daí o nome TensorFlow).

A idéia básica é a seguinte:

- Os dados são tabelas n-dimensionais. O caso mais comum é matrizes.

- A computação é definida como um grafo

- Um programa típico com TensorFlow consiste em duas partes: a definição do grafo, e a execução do grafo.

Vamos trabalhar melhor estes conceitos nesta aula.

TensorFlow foi criado para trabalhar com dados em larga escala: podemos construir modelos com milhões de parâmetros e bilhões de pontos de dados! Aliás, é para isso que a biblioteca foi criada pelo time de machine learning do Google (o projeto Google Brain), e é usado em serviços de larga escala como o Google Search, Google Photos, etc.

**Instalação**

```pip install tensorflow
```

**Documentação**

https://www.tensorflow.org/programmers_guide/


# Criando e executando um grafo

Vamos criar um grafo para executar a função $f(x) = x^2y + y + 2$.

In [2]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

print(x)
print(f)

/Users/gil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<tf.Variable 'x:0' shape=() dtype=int32_ref>
Tensor("add_1:0", shape=(), dtype=int32)


O código acima criou duas variáveis: `x` e `y`. A linha a seguir combina estas variáveis em uma expressão numérica. O truque está na sobrecarga dos operadores `*` e `+`, que combina as variáveis e constantes de modo a construir o grafo computacional.

Com este código a primeira parte da tarefa está completa! Vamos agora executar este grafo computacional. Para tanto precisamos criar uma sessão de execução de grafo (*Session*). O objeto da sessão é o instrumento de conexão entre a definição do grafo e os dispositivos computacionais onde este grafo será executado. Esta é uma grande sacada do TensorFlow: a separação da definição do modelo e a sua implementação computacional.

**Pergunta:** Por que você acha que é importante distinguir entre o modelo e seu ambiente computacional?

Vamos criar uma sessão e executar o grafo:

In [3]:
# Cria uma sessão.
sess = tf.Session()

# Antes de rodar o grafo temos que rodar os inicializadores.
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)

42


e agora temos que fechar a sessão:

In [4]:
sess.close()

Um modo melhor de escrever o mesmo código é usar o *keyword* `with`:

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


Para facilitar as coisas, TensorFlow mantem a noção de grafo default e sessão default. Assim, o código acima é equivalente à:

In [6]:
with tf.Session():
    tf.get_default_session().run(x.initializer)
    tf.get_default_session().run(y.initializer)
    result = f.eval()
    
print(result)

42


Para evitar o tédio de ter que inicializar cada variável, TensorFlow permite a criação de um nó no grafo voltado para a inicialização automática das variáveis globais:

In [7]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


# Grafos

As variáveis são criadas em grafos de execução. TensorFlow adiciona automaticamente as variáveis ao grafo default:

In [8]:
x.graph is tf.get_default_graph()

True

Toda vez que criamos uma variável ela é adicionada ao grafo default. Isso pode fazer com que tenhamos várias versões da mesma variável no grafo quando estamos trabalhando em um ambiente interativo como o Jupyter Notebook. Para "limpar" o grafo default:

In [9]:
tf.reset_default_graph()

Podemos criar variáveis em um grafo diferente, se assim quisermos:

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


Vamos construir uma função utilitária para resetar o grafo de execução:

In [11]:
def reset_graph(seed=RANDOM_SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Quando um nó é executado?

Considere a seguinte situação:

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


Neste exemplo o nó y é executado, o que requer a execução do nó x, que por sua vez requer a execução do nó w. Na linha seguinte o nó z é executado, que requer a **reexecução** do nó x, que requer a **reexecução** do nó w! TensorFlow não guarda os valores entre execuções.

Para garantir que o TensorFlow reaproveite os valores calculados, peça à sessão que calcule y e z na mesma execução:

In [13]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Regressão linear com Gradient Descent

Para ilustrar como o TensorFlow funciona, vamos otimizar os parâmetros de uma regressão linear usando o algoritmo Gradient Descent. 

Vamos usar os dados do dataset california housing para testar o TensorFlow. Vamos aplicar um StandardScaler nestes dados usando o scikit-learn (poderíamos fazer com TensorFlow, mas como isso é apenas uma demonstração vamos fazer assim mesmo).

In [14]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

Para construir um programa que otimiza os parâmetros de uma regressão linear, temos que fazer o seguinte:

Fase de construção do grafo:

- Construir o grafo da função de erro médio quadrático (MSE).
- Construir o gradiente desta função.
- Construir um nó que atualiza o valor dos parâmetros na direção contrária ao gradiente.

Fase de execução:
- Rodar a operação de atualização dos parâmetros até convergência ou até um número pré-definido de iterações.


In [15]:
reset_graph()

# Hiperparâmetros.
n_epochs = 2000
learning_rate = 0.01

# Fase de construção do grafo.

#
# Construindo o grafo do MSE:
#

# Dados originais (o conjunto de treinamento, claro).
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# Calcula a predição.
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# Calcula o MSE.
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#
# Construir o gradiente. Vamos usar o autodiff do TensorFlow.
#
gradients = tf.gradients(mse, [theta])[0]

#
# Construir o nó que atualiza os parâmetros.
#
training_op = tf.assign(theta, theta - learning_rate * gradients)

# Fase de execução do grafo.
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474
Epoch 1000 MSE = 0.53062946
Epoch 1100 MSE = 0.5294236
Epoch 1200 MSE = 0.5284622
Epoch 1300 MSE = 0.5276914
Epoch 1400 MSE = 0.5270721
Epoch 1500 MSE = 0.5265715
Epoch 1600 MSE = 0.5261664
Epoch 1700 MSE = 0.5258372
Epoch 1800 MSE = 0.52556866
Epoch 1900 MSE = 0.5253492
Best theta:
[[ 2.0685525e+00]
 [ 8.5735834e-01]
 [ 1.2673113e-01]
 [-3.1273845e-01]
 [ 3.4245059e-01]
 [-1.9602366e-03]
 [-4.0592730e-02]
 [-8.1566942e-01]
 [-7.8930181e-01]]


Vamos comparar com Scikit-learn:

In [17]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression(fit_intercept=False)
reg.fit(scaled_housing_data_plus_bias, housing.target.reshape(-1, 1))

print(reg.coef_)

[[ 2.06855817  0.8296193   0.11875165 -0.26552688  0.30569623 -0.004503
  -0.03932627 -0.89988565 -0.870541  ]]


## Usando otimizadores pré-definidos

TensorFlow já tem vários otimizadores de função prontos:

In [18]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


# Trabalhando com dados variáveis

Ao invés de colocar nossos dados em uma constante, podemos usar nós do tipo *placeholder* (espaço em branco) a serem preenchidos em tempo de execução. Assim nosso código pode ser reutilizado para vários experimentos.

Por exemplo, vamos construir um grafo que soma 5 aos dados, mas sem definir os dados de antemão em uma constante. Vamos, na verdade, preencher os dados em tempo de execução.

In [19]:
reset_graph()

# Constroi o grafo de execução com um placeholder de 3 colunas e número indefinido de linhas.
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

# Fase de execução.
with tf.Session() as sess:
    # Ao rodar um nó que está conectado a um placeholder, precisamos definir o valor do placeholder.
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    
    # Mesma coisa aqui, ressaltando que podemos mudar o número de linhas de A pois 
    # na definição de A não fixamos o número de linhas.
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


# Implementando mini-batch gradient descent

In [20]:
# Hiperparâmetros.
n_epochs = 10
learning_rate = 0.01

# Construindo o grafo de execução. Note que estamos usando placeholders para X e y.
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Fase de execução.
init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            # Escolhe os dados do batch.
            np.random.seed(epoch * n_batches + batch_index)
            indices = np.random.randint(m, size=batch_size)
            X_batch = scaled_housing_data_plus_bias[indices]
            y_batch = housing.target.reshape(-1, 1)[indices]
            
            # Roda um passo do Gradient Descent para este batch.
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    
print(best_theta)

[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


# Salvando modelos

Salvar modelos é uma boa ideia: pode servir para reiniciar um processo de treinamento que caiu, ou para visualizar o progresso de um processo de treinamento, ou para carregar um modelo treinado para usar em produção. Eis como fazê-lo:

In [22]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
    
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


Agora podemos resgatar o modelo do disco:

In [23]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()
    
print(best_theta_restored)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


Note que o código acima funcionou porque já tinhamos a variável theta, e o que foi carregado foram os valores numéricos dos tensores. O TensorFlow também grava o grafo de execução no arquivo .meta, eis como carrega-lo:

In [24]:
# Vamos começar com um grafo limpo:
reset_graph()

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

print(best_theta_restored)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


# TensorBoard

TensorFlow vem com uma ferramenta de visualização do progresso do treinamento chamada TensorBoard.

Vamos inicialmente criar um diretório para armazenar os logs do TensorFlow:

In [25]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "/tmp/tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [26]:
# Hiperparâmetros.
n_epochs = 10
learning_rate = 0.01

# Construindo o grafo de execução. Note que estamos usando placeholders para X e y.
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Vamos criar dois objetos extras: um para gerar um sumário do treinamento (neste caso contendo 
# apenas o MSE) e outro para escrever o log no disco.
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# Fase de execução.
init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            # Escolhe os dados do batch.
            np.random.seed(epoch * n_batches + batch_index)
            indices = np.random.randint(m, size=batch_size)
            X_batch = scaled_housing_data_plus_bias[indices]
            y_batch = housing.target.reshape(-1, 1)[indices]

            # A cada 10 batches grava o sumário no disco.
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            # Roda um passo do Gradient Descent para este batch.
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

# Tem que fechar o arquivo na mão.
file_writer.close()

print(best_theta)

[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


Agora abra um terminal no diretório /tmp e rode o seguinte comando:

```tensorboard --logdir tf_logs/
```

Abra no browser a página http://127.0.0.1:6006

(É 6006 porque é GOOG...)

Isto foi uma introdução ao TensorFlow. Continue estudando o capítulo 9 para conhecer mais detalhes finos como NameScopes, modularização, etc!